# Sinus approximator

## load and import libraries

In [1]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.jetbrains.kotlinx:kotlinx-serialization-json:1.6.3")
    }
}

In [2]:
class Neuron(val weights: FloatArray, val bias: Float, val activation: (Float) -> Float) {

    fun forward(inputs: FloatArray): Float {
        if (weights.isNotEmpty()) {
            var sum = 0.0f
            inputs.forEachIndexed {i, inpValue ->
                sum += inpValue * weights[i]
            }
            return activation(sum + bias)
        } else {
            return inputs.fold(0.0f) { acc, input -> acc + input }
        }
    }

    override fun toString(): String {
        return weights.joinToString(prefix = "[", postfix = "]")
    }
}

In [3]:
data class Layer(val neurons: List<Neuron>)

class DenseNet(private val layers: List<Layer>) {

    fun forward(inputData: FloatArray): FloatArray {
        var input = inputData
        layers.forEach { layer ->
            if (layer.neurons.isNotEmpty()) {
                val layerResult = FloatArray(layer.neurons.size)
                layer.neurons.forEachIndexed { index, neuron: Neuron ->
                    layerResult[index] = neuron.forward(input)
                }
                input = layerResult
            }
        }
        return input
    }

    override fun toString(): String {
        return layers.joinToString(prefix = "[", postfix = "]")
    }

}

## Load trained data

In [4]:
import kotlinx.serialization.json.Json
import java.io.File
import kotlinx.serialization.Serializable

In [5]:
@Serializable
data class ArrayValues(
    val unique_parameter_name: String,
    val array_values: ArrayValue
)

@Serializable
data class ArrayValue(
    val values: List<Double>
)

In [6]:
fun loadWeights(jsonFile: File):List<ArrayValues> {
    // Example: Loading JSON from a file
    var jsonString = ""
    try {
        jsonString = jsonFile.readText(Charsets.UTF_8)
    } catch (oome: OutOfMemoryError) {
        //Log the info
        System.err.println("Array size too large")
        System.err.println("Max JVM memory: " + Runtime.getRuntime().maxMemory())
    }

    // Initialize Json object
    val json = Json { ignoreUnknownKeys = true }

    // Deserialize JSON to Kotlin objects
    return json.decodeFromString(jsonString)
}

In [7]:
val weightAndBiases = loadWeights(File("sinus_aproximator_weights.json"))

In [8]:
fun List<ArrayValues>.getBy(name: String): FloatArray {
    firstOrNull { neuron -> neuron.unique_parameter_name == name }?.let {
        return@getBy FloatArray(it.array_values.values.size) { i -> it.array_values.values[i].toFloat() }
    }
    return FloatArray(0)
}

## Create network

### Input

In [9]:
val neuronsInHiddenLayer = 16

In [10]:
val input = Layer(
    listOf(
        Neuron(
            FloatArray(0),
            0.0f,
            { value -> value }
        )
    )
)

### Hidden

In [11]:
val relu = { input: Float -> if (input < 0) 0.0f else input }

In [12]:
var hidden1 = Layer(
    List(16) { index ->
        Neuron(
            weightAndBiases.getBy("layer1.weight").copyOfRange(index, index + 1),
            weightAndBiases.getBy("layer1.bias")[index],
            relu
        )
    }
)

In [20]:
val allWeights = weightAndBiases.getBy("layer2.weight")
val hidden2 = Layer(

    List(neuronsInHiddenLayer) { index ->
        val neuronWeights = allWeights.copyOfRange(
            index * neuronsInHiddenLayer,
            index * neuronsInHiddenLayer + neuronsInHiddenLayer
        )
        Neuron(
            neuronWeights,
            weightAndBiases.getBy("layer2.bias")[index],
            relu
        )
    }
)

In [21]:
var output = Layer(
    listOf(
        Neuron(
            weightAndBiases.getBy("output_layer.weight"),
            weightAndBiases.getBy("output_layer.bias")[0],
            relu
        )
    )
)

In [22]:
val nn = DenseNet(listOf(input, hidden1, hidden2, output))

## Validation

In [23]:
%use kandy
%use dataframe

In [24]:
val x_values = List(100) { index ->
    (index / (100 - 1).toFloat()) * (PI / 2)
}

val y_values = List(100) { index ->
    sin(x_values[index])
}

val y_nn_values = List(100) { index ->
    nn.forward(floatArrayOf(x_values[index].toFloat()))[0]
}

val error = List(100) { index ->
    abs(y_values[index] - y_nn_values[index])
}

val df = dataFrameOf(
    "y" to y_values+ y_nn_values + error,
    "x" to x_values + x_values + x_values,
    "mode" to List(100) { "sin" } + List(100) { "nn" } + List(100) { "error" }
)

In [25]:
df.plot {
    line {
        x("x")
        y("y")
        color("mode") {
            scale = categorical("sin" to Color.PURPLE, "nn" to Color.ORANGE, "error" to Color.RED)
        }
        width = 1.5
    }
}//.save("result.svg")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="zRe1Vd"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"mode":["sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","sin","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","nn","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error","error"],
"x":[0.0,0.015866629548807864,0.03173325909761573,0.04759989010934167,0.06346651819523146,0.07933314628112124,0.09519978021868333,0.11106640245290081,0.1269330363904629,0.142799670328025,0.1586662925622425,0.17453292649980456,0.19039956043736667,0.20626618267158414,0.22213280490580162,0.23799945054670835,0.2538660727809258,0.2697326950151433,0.28559934065605,0.3014659628902675,0.317332585124485,0.3331992307653917,0.3490658529996091,0.3649324752338266,0.38079912087473333,0.39666571970226155,0.4125323653431683,0.428399010984075,0.44426560981160323,0.46013225545250996,0.4759989010934167,0.4918654999209449,0.5077321455618516,0.5235987912027583,0.5394653900302866,0.5553320356711933,0.5711986813121,0.5870652801396282,0.602931925780535,0.6187985714214417,0.63466517024897,0.6505318158898766,0.6663984615307834,0.6822650603583116,0.6981317059992183,0.713998351640125,0.7298649504676532,0.74573159610856,0.7615982417494667,0.7774648405769949,0.7933314394045231,0.8091980850454299,0.8250647306863366,0.8409313763272434,0.85679802196815,0.8726646676090568,0.8885312196232065,0.9043978652641131,0.9202645109050199,0.9361311565459266,0.9519978021868334,0.9678644478277402,0.9837309998418898,0.9995976454827965,1.0154642911237033,1.03133093676461,1.0471975824055166,1.0630641344196663,1.0789307800605732,1.0947974257014799,1.1106640713423865,1.1265307169832934,1.1423973626242,1.1582639146383498,1.1741305602792564,1.1899972059201631,1.20586385156107,1.2217304972019767,1.2375971428428834,1.253463694857033,1.26933034049794,1.2851969861388466,1.3010636317797533,1.31693027742066,1.3327969230615668,1.3486634750757165,1.3645301207166232,1.3803967663575298,1.3962634119984365,1.412130057639